In [1]:
import numpy as np
import mne
from mne.decoding import CSP
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier  # Neural Network Classifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from mne.filter import filter_data
import matplotlib.pyplot as plt

In [2]:
# Define the path to the EDF file
edf_file = '../data/raw data/S001/S001R04.edf'

# Load the raw EEG data
raw = mne.io.read_raw_edf(edf_file, preload=True)

# Display basic information about the loaded data
print(raw.info)

# Plot raw EEG data interactively (optional)
# raw.plot(duration=30, n_channels=30, scalings='auto', title='Raw EEG Signals', show=True)

# Load the raw EEG data with annotations
raw = mne.io.read_raw_edf(edf_file, preload=True)

# Access annotations
annotations = raw.annotations

# Display annotations
print(annotations)

# Define event dictionary
event_dict = {
    'Rest': 1,                # T0
    'Imagining Left Hand': 2,  # T1
    'Imagining Right Hand': 3  # T2
}

# Convert annotations to events and event_id
events, event_id = mne.events_from_annotations(raw)

# Update event_id with descriptive labels
event_id = {
    'Rest': 1,
    'Imagining Left Hand': 2,
    'Imagining Right Hand': 3
}

print(event_id)

from mne import Epochs

# Define epoch parameters
tmin = -0.2  # 200 ms before the event
tmax = 3.8   # 3800 ms after the event
baseline = (None, 0)  # Baseline correction

# Create epochs
epochs = Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=baseline, preload=True)

# Display epoch information
print(epochs)

# Plot an average epoch for 'Imagining Left Hand' (optional)
# fig = epochs['Imagining Left Hand'].average().plot()
# fig.suptitle('Imagining Left Hand')
# plt.show()

# Plot an average epoch for 'Imagining Right Hand' (optional)
# fig = epochs['Imagining Right Hand'].average().plot()
# fig.suptitle('Imagining Right Hand')
# plt.show()

# Plot an average epoch for 'Rest' (optional)
# fig = epochs['Rest'].average().plot()
# fig.suptitle('Rest')
# plt.show()

# Select only 'Imagining Left Hand' and 'Imagining Right Hand' epochs
motor_epochs = epochs[['Imagining Left Hand', 'Imagining Right Hand']]

# Extract labels: 0 for Left, 1 for Right
labels = motor_epochs.events[:, -1] - 2  # 'Imagining Left Hand' = 2, 'Imagining Right Hand' = 3

# Verify the distribution of classes
print("Class distribution:", np.bincount(labels))

# Get the data as a NumPy array: shape (n_epochs, n_channels, n_times)
X = motor_epochs.get_data()
y = labels

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

Extracting EDF parameters from /Users/helechuan/Library/Mobile Documents/com~apple~CloudDocs/Documents/4th Year University/4th Year Project/64-Channels-EEG-Data-ML-training/data/raw data/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
<Info | 8 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
 subject_info: 3 items (dict)
>
Extracting EDF parameters from /Users/helechuan/Library/Mobile Documents/com~apple~CloudDocs/Documents/4th Year University/4th Year Project/64-Channels-EEG-Data-ML-training/data/raw data/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
<Annotations 

In [3]:
# Define frequency bands (in Hz)
freq_bands = [
    (8, 12),    # Alpha
    (13, 20),   # Beta 1
    (21, 30),   # Beta 2
    (31, 40),   # Low Gamma
    (41, 50),   # High Gamma
    (51, 60),   # Very High Gamma
    (61, 70),   # Ultra High Gamma
    (71, 79)    # Max Gamma
]

In [9]:
class FBCSP(BaseEstimator, TransformerMixin):
    def __init__(self, freq_bands, n_csp=4, sfreq=160):
        """
        Initialize the FBCSP transformer.
        
        Parameters:
        - freq_bands: List of tuples defining frequency bands.
        - n_csp: Number of CSP components per band.
        - sfreq: Sampling frequency of the EEG data.
        """
        self.freq_bands = freq_bands
        self.n_csp = n_csp
        self.sfreq = sfreq
        self.csp_list_ = []  # To store CSP models for each band

    def fit(self, X, y):
        """
        Fit CSP models for each frequency band.
        
        Parameters:
        - X: EEG data, shape (n_trials, n_channels, n_times)
        - y: Labels, shape (n_trials,)
        
        Returns:
        - self
        """
        self.csp_list_ = []  # Reset in case of multiple fits
        for band in self.freq_bands:
            print(f'Fitting CSP for band: {band[0]}-{band[1]} Hz')
            # Band-pass filter the data for the current frequency band
            X_filtered = np.array([
                filter_data(trial, sfreq=self.sfreq, l_freq=band[0], h_freq=band[1], method='iir', verbose=False)
                for trial in X
            ])
            
            # Initialize and fit CSP
            csp = CSP(n_components=self.n_csp, reg=None, log=True, norm_trace=False)
            csp.fit(X_filtered, y)
            self.csp_list_.append(csp)
        return self

    def transform(self, X):
        """
        Transform EEG data using the fitted CSP models.
        
        Parameters:
        - X: EEG data, shape (n_trials, n_channels, n_times)
        
        Returns:
        - X_features: Extracted features, shape (n_trials, n_bands * n_csp)
        """
        features = []
        for idx, band in enumerate(self.freq_bands):
            print(f'Transforming data for band: {band[0]}-{band[1]} Hz')
            csp = self.csp_list_[idx]
            # Band-pass filter the data for the current frequency band
            X_filtered = np.array([
                filter_data(trial, sfreq=self.sfreq, l_freq=band[0], h_freq=band[1], method='iir', verbose=False)
                for trial in X
            ])
            
            # Apply CSP transformation
            X_csp = csp.transform(X_filtered)
            
            # Extract log-variance features
            X_logvar = np.log(np.var(X_csp, axis=1))
            
            # Reshape X_logvar to (n_trials, 1) to make it 2D
            X_logvar = X_logvar.reshape(-1, 1)
            
            # Append the reshaped feature
            features.append(X_logvar)
        
        # Concatenate features from all bands along axis=1
       
        X_features = np.column_stack(features)
        return X_features

In [10]:
# Initialize FBCSP transformer
fbcsp = FBCSP(freq_bands=freq_bands, n_csp=4, sfreq=160)  # Replace sfreq with your actual sampling frequency

# Initialize Neural Network classifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

# Define the classification pipeline
pipeline = Pipeline([
    ('fbcsp', fbcsp),
    ('classifier', mlp)
])

In [11]:
# Define cross-validation strategy
n_splits = 5  # Adjust based on your dataset size
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Perform cross-validation
print("Starting Cross-Validation...")
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')

print(f"\nCross-Validation Accuracy Scores: {scores}")
print(f"Mean Accuracy: {scores.mean() * 100:.2f}%")
print(f"Standard Deviation: {scores.std() * 100:.2f}%")

Starting Cross-Validation...
Fitting CSP for band: 8-12 Hz
Computing rank from data with rank=None
    Using tolerance 8.6e-05 (2.2e-16 eps * 64 dim * 6.1e+09  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Fitting CSP for band: 13-20 Hz
Computing rank from data with rank=None
    Using tolerance 8.2e-05 (2.2e-16 eps * 64 dim * 5.8e+09  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Fitting CSP for band: 21-30 Hz
Computing rank from data with rank=None
    Using tolerance 5.5e-05 (2.2e-16 eps * 64 dim * 3.9e+09  max singular value)
    Estimated rank (data): 64
    data: rank 64 

/Users/helechuan/Library/Mobile Documents/com~apple~CloudDocs/Documents/4th Year University/4th Year Project/64-Channels-EEG-Data-ML-training/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Transforming data for band: 31-40 Hz
Transforming data for band: 41-50 Hz
Transforming data for band: 51-60 Hz
Transforming data for band: 61-70 Hz
Transforming data for band: 71-79 Hz
Fitting CSP for band: 8-12 Hz
Computing rank from data with rank=None
    Using tolerance 8.6e-05 (2.2e-16 eps * 64 dim * 6.1e+09  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Fitting CSP for band: 13-20 Hz
Computing rank from data with rank=None
    Using tolerance 8.1e-05 (2.2e-16 eps * 64 dim * 5.7e+09  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Fitting CSP for band: 21-30 Hz
Computing ran

/Users/helechuan/Library/Mobile Documents/com~apple~CloudDocs/Documents/4th Year University/4th Year Project/64-Channels-EEG-Data-ML-training/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Transforming data for band: 31-40 Hz
Transforming data for band: 41-50 Hz
Transforming data for band: 51-60 Hz
Transforming data for band: 61-70 Hz
Transforming data for band: 71-79 Hz
Fitting CSP for band: 8-12 Hz
Computing rank from data with rank=None
    Using tolerance 8.6e-05 (2.2e-16 eps * 64 dim * 6e+09  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Fitting CSP for band: 13-20 Hz
Computing rank from data with rank=None
    Using tolerance 7.9e-05 (2.2e-16 eps * 64 dim * 5.5e+09  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Fitting CSP for band: 21-30 Hz
Computing rank 

/Users/helechuan/Library/Mobile Documents/com~apple~CloudDocs/Documents/4th Year University/4th Year Project/64-Channels-EEG-Data-ML-training/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Transforming data for band: 21-30 Hz
Transforming data for band: 31-40 Hz
Transforming data for band: 41-50 Hz
Transforming data for band: 51-60 Hz
Transforming data for band: 61-70 Hz
Transforming data for band: 71-79 Hz
Fitting CSP for band: 8-12 Hz
Computing rank from data with rank=None
    Using tolerance 8.4e-05 (2.2e-16 eps * 64 dim * 5.9e+09  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Fitting CSP for band: 13-20 Hz
Computing rank from data with rank=None
    Using tolerance 8.1e-05 (2.2e-16 eps * 64 dim * 5.7e+09  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Fitting

/Users/helechuan/Library/Mobile Documents/com~apple~CloudDocs/Documents/4th Year University/4th Year Project/64-Channels-EEG-Data-ML-training/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
